# Bayesian decoding of neural activity

<div class="warning" style='padding:0.5em;background-color:#f1f1f1;border:1px solid black;width:95%'>

Lesson time: 60 m   
Contributors: Federico Stella, Davide Spalla

---
### In this lesson you will learn:
- What a bayesian decoder is and how to use it
- How to decode the position of an animal running in one and two dimensions
- How to use decoding techniques to investigate neural activity during sleep

</div>


## Introduction
---
 
In this lesson we will take a look at some techniques aimed at "reading" neural activity and allowing us to interpret its content and to evaluate how much information it carries about a specific set of stimuli, conditions or states. 



## Naive Bayes decoding
---
 Bayesian decoders are a simple but powerful tool for decoding the value of a stimulus from the observed neural activity.
 Let $P(s)$ denote the probability of presentation of stimulus $s$ (belonging to a set $S$) and $P(r|s)$ denote the conditional probability of obtaining a population response $r$ (out of a response set $R$) when stimulus s is presented. Using Bayes' theorem, we can write the probability of the stimulus given the response:

 $$P(s|r) = \frac{P(r|s)P(s)}{P(r)}$$

This equation gives the posterior probability that, given a response $r$, stimulus $s$ was presented. 
Bayesian decoding calculates from this posterior probability distribution a single prediction of the most likely stimulus, i.e. the maximum of the posterior distribution. 

In the case of a neural population, $P(r|s)$ lives in an $N$ dimensional space, where $N$ is the number of observed neuron, and its estimation can be a daunting problem. We can, however, make the simplifying assumption that the activity of each neuron is independent.In this case, we can factorize likelihood as:

$$P(r|s)=\prod_{i=1}^{N}P(r_{i}|s)$$

Note that, since we are generally interested in finding the maximum of the posterior distribtuion, we often do not need to evaluate the denominator $P(r)$ (which is usually difficult to calculate).
Moreover, if the prior $P(s)$ is flat (each stimulus value is equiprobable a priori), the problem simplifies further, and we are left with the follwing expression for the decoded stimulus value:

$$s^{*}=argmax_{s}(\prod_{i=1}^{N}P(r_{i}|s))$$

## more on poisson cell, log likelihood and final formula




### Decoding position on a linear track
---

As an example, let's look at how we can decode the position of the animal on a linear track from the activity of its hippocampal place cells. We will start from simulated data for a clearer example, then move on to some real data.
For the simulated data, well' use some generated with the code you can find in the lesson on [place cells]() of this course.

First, let's import the data:



In [35]:
#code: data import

To perform bayesian decoding, we need to estimate $P(r_{i}|s)$. This is done by computiong the firing rate maps of each cell:

In [ ]:
## ratemap code

Now we can apply the equations we derived, to decode the position of the animal at each time

In [ ]:
# code decoding
x_decoded=zeros(size(spikes,2),1);
for t_bin=1:size(spikes,2)
    
    if(sum(spikes(:,t_bin),1)>0) % Check if the time window contains spikes
    
    Post_p=NaN(size(estim_firing_rate,2),1);
    for i=1:size(estim_firing_rate,2)
        % Note that we work with log so that we can sum probabilities
        % instead of multiplying them 
        Post_p(i)=sum(log(poisspdf(spikes(:,t_bin),estim_firing_rate(:,i)/sampling_rate)));
       
    end

    x_decoded(t_bin) = find(Post_p==nanmax(Post_p),1,'first');
    else
    x_decoded(t_bin) = NaN;   
    end
end


Let's now look at the results. Let's plot the decoded trajectory against the true one, and look at the distribution of errors

### Decoding position in 2 dimensions
---


In [ ]:
# code

## Sequential reactivation during sleep
---

The use of decoding methods is not limited to measure the amount of information that a neural population carries about a specific set of stimuli, or to predict what stimulus is present at a given time. It is extremely useful to interpret the content of neural activity when no stimulus is present and such activity is spontaneously generated by the network and is not elicited by a specific stimulus. 
In the case of the hippocampus it has now been extensively observed that during rest periods (either while the animal is standing still, waiting somewhere, or while it is sleeping) place cells reactivate specific patterns of activity corresponding to sequences of locations in the environment. This phenomenon, called "replay" is thought to be critical for the formation of memory, for the acquisition of novel knowledge and for the reorganization of different sorts of information in the brain (You can read [this review](https://www.sciencedirect.com/science/article/abs/pii/S0166223610000172?via%3Dihub) for an overview on this vast and ebullient field of research).    
Here we want simulate the occurrence of such reactivation events. Some of them will actually contain sequences, that is, the neural activity will represents a series of locations in the order they appear on the linear track. Other reactivation events will instead contain activity representing locations of the linear track, but these locations will be randomly drawn, so they will present no clear ordering. 

In [ ]:
spikes_react_events = cell(200,1); %200 reactivation events

for event=1:200

sampling_rate = 100;

t_react = 1:100;



noise_x_react=5; % Noise in the reactivation of the sequence
noise_t_react=5; % Noise in the timing of the spikes 

if(event<=100)
    % Generate "real" sequences for the first half of events
    x_start = rand(1)*track_length; % Starting point
    x_end = rand(1)*track_length;% Ending point
    x_react = ceil(linspace(x_start,x_end,100)+randn(1,100)*noise_x_react)';
elseif(event>100)
    % Pick locations randomly for the second half 
    x_react = ceil(rand(100,1)*track_length);
end

x_react(x_react<1)=1;
x_react(x_react>track_length)=track_length;

noise_firing_rate = 0.1; % the baseline noise firing rate

spikes_react = zeros(n_cells,numel(t_react));

% Generate spikes according to the location being reactivated in this event
for i=1:n_cells
    inst_rate = true_firing_rate(i,x_react) + randn(1,numel(x_react))*noise_firing_rate;
    inst_rate(inst_rate<0) = 0;
    spikes_loc = find(poissrnd(inst_rate/sampling_rate));
    spikes_loc = spikes_loc + round(randn(numel(spikes_loc),1)*noise_t_react);
    spikes_loc = spikes_loc(ismember(spikes_loc,1:100));
    spikes_react(i,spikes_loc)=1;
    
end
% Put the data in a cell-array
spikes_react_events{event} = spikes_react;

end


% Reorder cells in terms of the position of their place field on the track 
[~,pf_order]=sort(pf_centers,'ascend');

%We can use this reordering to "visualize the reactivation"
% An example of a sequential reactivation 
Event_Spikes=spikes_react_events{5};
spy(Event_Spikes(pf_order,:))

% An example of a non-sequential reactivation
Event_Spikes=spikes_react_events{102};
spy(Event_Spikes(pf_order,:))

We can now use bayesian decoding to detect reactivated sequences among these events.

In [ ]:
t_resize = 10; % We use spikes from multiple time windows for the decoding

reactivation_slope = zeros(200,1);
for event=1:200

    spikes_react = spikes_react_events{event};
    

    spikes_react_sampled = zeros(n_cells,size(spikes_react,2)/t_resize);

% We generate a new spike matrix with the re-sized window
for t_r=1:100/t_resize
spikes_react_sampled(:,t_r) = sum(spikes_react(:,(t_r-1)*10+1:(t_r)*10),2);
end

%And then we perform the same Bayesian decoding as before 
x_decoded=zeros(size(spikes_react_sampled,2),1);
for t_bin=1:size(spikes_react_sampled,2)
    
    if(sum(spikes_react_sampled(:,t_bin),1)>0) % Check if the time window contains spikes
    
    Post_p=NaN(size(estim_firing_rate,2),1);
    for i=1:size(estim_firing_rate,2)
        
        Post_p(i)=sum(log(poisspdf(spikes_react_sampled(:,t_bin),estim_firing_rate(:,i)/sampling_rate*t_resize)));
       
    end

    x_decoded(t_bin) = find(Post_p==nanmax(Post_p),1,'first');
    else
    x_decoded(t_bin) = NaN;   
    end
end

% Linear fit of 
%beta = polyfit(1:numel(x_decoded),x_decoded',1);
[rr,pp] = corr((1:numel(x_decoded))',x_decoded);

reactivation_slope(event) = pp;%beta(1);


end


<div class="warning" style='padding:0.5em; background-color:#f1f1f1;border:1px solid black;width:95%'>

### Key points 

- Naive Bayes decoders use bayes' formula to estimate the probability of observing a stimulus given a certain pattern of neural responses
- In its simplest form, this algorithms relies on the assumption of *flat stimulus prior*, *and independence between neurons*
- This decoder can be used to recover the animal position during 1D and 2D navigation
- Decoding approaches can be used to investigate spontaneous neural activity during sleep or rest.

<div class="warning" style='padding:0.5em; background-color:#f1f1f1;border:1px solid black;width:95%'>

### References and resources

**Books & papers**
* Review on neural decoding: https://www.nature.com/articles/nrn2578
* Review on information processing in neural populations: https://www.nature.com/articles/35039062 
* Review on hippocampal replay and reactivation: https://www.sciencedirect.com/science/article/abs/pii/S0166223610000172?via%3Dihub 

**Websites & blogposts**

**Software**
* A python package for neural decoding: https://github.com/KordingLab/Neural_Decoding



## Exercises
You can find the exercises for this lessons in [exercises.ipynb](./exercises.ipynb)